In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import matplotlib.pylab as pylab
import cartopy.feature as cfeature
from endgame import hist2d
import scipy.stats as stats
import xarray as xr
import ipdb
import glob
import itertools
import numpy.ma as ma
from utils import u_statistics as u_stat
from scipy.stats import gaussian_kde
import pickle as pkl
from utils import u_plot as uplot
import pandas as pd
from scipy.stats import gaussian_kde, linregress
import matplotlib.cm as cm
from utils import u_met
import seaborn
import metpy
from metpy import calc
from metpy.units import units
import statsmodels.api as sm
from scipy import ndimage
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from scipy.stats import binned_statistic_2d

from numpy.polynomial import polynomial as P

##for regridding, install xesmf:
# conda install esmpy
# pip install xesmf
from utils import constants as cnst

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


Loads bulk statistics derived from having cut out storms (saved in MCSfiles), OBS includes rainfall from TRMM radar

In [3]:
def create_dic(dic):
    
    tt = np.array(dic['tmin'])
    mask = np.where(tt<=-50)
    
    pp = np.array(dic['pmax'])[mask] 
    sh = np.array(dic['shear'])[mask]  # 925-650 hPa shear
    umin = np.array(dic['u650'])[mask] # minimum 650hPa zonal wind 
    umax = np.array(dic['u925'])[mask] # maximum 925hPa zonal wind
    qq = np.array(dic['q925'])[mask]   # 925hPa spec humidity
    qup = np.array(dic['q650'])[mask]  # 650hPa spec humidity
    tlow = np.array(dic['t925'])[mask] # 925hPa temperature
    tt = np.array(dic['tmin'])[mask]   # minimum cloud top temperature
    ttm = np.array(dic['tmean'])[mask] # mean cloud top temperature
    month = np.array(dic['month'])[mask] 
    hour = np.array(dic['hour'])[mask]
    year = np.array(dic['year'])[mask]
    day = np.array(dic['day'])[mask]
    date = np.array(dic['date'])[mask]
    area = np.array(dic['area'])[mask] # -50C storm area
    lat = np.array(dic['clat'])[mask] # location of minimum temperature
    lon = np.array(dic['clon'])[mask] # location of minimum temperature
    tcwv = np.array(dic['tcwv'])[mask] # total column water vapour
    rh = np.array(dic['rh925'])[mask] # 925hPa relative humidity
    #pall = np.array(dic['p'])[mask] # precipitation field within -50C cloud area


    obs_pcp = {
        'prcp' : pp,
        'q925' : qq*1000,
        'q650' : qup*1000,
        't925' : tlow-273.15,
        'shear' : sh*-1,
        'month' : month,    
        'hour' : hour,
        'year' : year,
        'day' : day,
        'date' : date,
        'area' : (area)*(5**2),
        'tmin' : tt,
        'rh' : rh,
        'tcwv' : tcwv,
        'lat' : lat,
        'lon' : lon,
        #'pall' : pall
    } 
    
    obs = pd.DataFrame(obs_pcp)
    return obs

In [4]:
season = {#'DJF' : [1,5,9],
          'MAM' : [7,12,3],
           'JJA' : [11,4,8],
          'SON' : [10,6,9]}

In [5]:
file_path = cnst.CLOVER_SAVES + 'bulk_-50_5000km2_GPM_5-26N_16W17E_p15_ERA0.7_TCWV_hourly_wDate_*.p'

In [6]:
node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))

In [7]:
#shows available variables in raw dataset
testread = pkl.load( open (cnst.CLOVER_SAVES + 'bulk_-50_5000km2_GPM_5-26N_16W17E_p15_ERA0.7_TCWV_hourly_wDate_2004.p', 'rb'))
print(testread.keys())
del testread

dict_keys(['hour', 'month', 'year', 'day', 'date', 'area', 'lon', 'lat', 'clon', 'clat', 'minlon', 'minlat', 'tmin', 'tmean', 'tcwv', 'pmax', 'pmean', 'q925', 'q650', 'u925', 'u650', 'v925', 'v650', 'w925', 'w650', 'rh925', 'rh650', 't925', 't650', 'div925', 'div650', 'pv925', 'pv650', 'shear', 'pgt30', 'pgt01isvalid', 't', 'p', 'cape', 't2m'])


In [8]:
ds = glob.glob(file_path)

for ids, dss in enumerate(ds):
    data = pkl.load( open (dss, 'rb'))

    obs_gpmf_h = create_dic(data)
    if ids > 0:
        obs_gpmf = pd.concat([obs_gpmf, obs_gpmf_h])
    else:
        obs_gpmf = obs_gpmf_h

    del data

In [9]:
obs = obs_gpmf

In [10]:
obs

,prcp,q925,q650,t925,shear,month,hour,year,day,date,area,tmin,rh,tcwv,lat,lon
0,4.546894,15.324332,6.423853,24.874475,14.258469,7.0,17.0,2004.0,22.0,2004-07-22 17:30:00,5975.0,-77.805445,72.686592,47.977135,16.581171,0.312186
1,43.385965,14.628493,5.576330,22.627802,17.359905,9.0,19.0,2004.0,21.0,2004-09-21 19:30:00,83175.0,-75.000000,78.363197,47.305939,11.289606,-13.870493
2,20.520223,14.666178,6.573158,19.529047,4.076474,9.0,18.0,2004.0,15.0,2004-09-15 18:30:00,60150.0,-76.466973,95.035973,53.804394,6.743863,-7.025343
3,8.098122,15.512402,5.839916,23.767664,5.012726,8.0,17.0,2004.0,26.0,2004-08-26 17:30:00,7275.0,-73.380482,77.722290,50.499004,8.643763,10.914400
4,20.549192,14.712868,4.529143,22.579614,11.788300,9.0,18.0,2004.0,11.0,2004-09-11 18:30:00,6725.0,-76.211019,79.081085,42.870922,7.967304,-9.138874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2406,18.293368,15.889853,5.321009,22.611871,14.178018,5.0,21.0,2015.0,22.0,2015-05-22 21:00:00,6300.0,-62.783486,85.002289,55.583439,10.845026,-13.990753
2407,7.232957,15.499491,6.174738,27.257166,7.789907,6.0,20.0,2015.0,26.0,2015-06-26 20:30:00,15375.0,-78.557426,63.109238,44.427227,12.215550,-1.446545
2408,15.779857,14.543398,4.898322,27.249353,18.221504,6.0,21.0,2015.0,30.0,2015-06-30 21:00:00,18750.0,-69.710906,59.499866,42.788399,12.751741,0.816761
2409,29.401626,14.361146,4.938281,28.561853,17.353197,6.0,19.0,2015.0,28.0,2015-06-28 19:00:00,24975.0,-77.269752,54.105953,42.908611,12.697603,1.690304


In [12]:
node_days['node1'].values

array(['1981-01-01T00:00:00.000000000', '1981-01-02T00:00:00.000000000',
       '1981-01-03T00:00:00.000000000', ...,
       '2020-11-25T00:00:00.000000000', '2020-12-09T00:00:00.000000000',
       '2020-12-12T00:00:00.000000000'], dtype='datetime64[ns]')

In [13]:
print(np.max(obs_gpmf.lat))
print(np.min(obs_gpmf.lat))
print(np.max(obs_gpmf.lon))
print(np.min(obs_gpmf.lon))

17.824957782145926
5.163266295264673
12.107882421128927
-16.97683096799293


In [14]:
node_mcs_days = {}
for n in range(1,13):
    nstr = 'node'+str(n)
    node_mcs_days[nstr] = []
    nday = node_days[nstr]
    for ndate in nday:
        posis = ((ndate.year) == obs['year']) & ((ndate.month) == obs['month']) & ((ndate.day) == obs['day'])
        
        node_mcs_days[nstr].extend(obs['date'].values[posis])

In [15]:
obs_ind = obs.set_index('date')

In [17]:
node_mcs_days

{'node1': [numpy.datetime64('2004-03-11T20:00:00.000000000'),
  numpy.datetime64('2004-03-11T20:00:00.000000000'),
  numpy.datetime64('2004-03-11T21:00:00.000000000'),
  numpy.datetime64('2004-03-11T19:00:00.000000000'),
  numpy.datetime64('2004-11-20T21:00:00.000000000'),
  numpy.datetime64('2004-11-20T21:00:00.000000000'),
  numpy.datetime64('2004-11-20T19:00:00.000000000'),
  numpy.datetime64('2004-11-20T21:00:00.000000000'),
  numpy.datetime64('2004-11-20T21:00:00.000000000'),
  numpy.datetime64('2004-11-20T21:00:00.000000000'),
  numpy.datetime64('2004-11-21T19:00:00.000000000'),
  numpy.datetime64('2004-11-21T19:00:00.000000000'),
  numpy.datetime64('2004-11-21T21:00:00.000000000'),
  numpy.datetime64('2004-11-21T21:00:00.000000000'),
  numpy.datetime64('2004-11-21T15:00:00.000000000'),
  numpy.datetime64('2004-11-21T19:00:00.000000000'),
  numpy.datetime64('2004-11-21T17:00:00.000000000'),
  numpy.datetime64('2004-11-21T15:00:00.000000000'),
  numpy.datetime64('2004-11-21T21:00:

In [16]:
obs_ind

,prcp,q925,q650,t925,shear,month,hour,year,day,area,tmin,rh,tcwv,lat,lon
date,,,,,,,,,,,,,,,
2004-07-22 17:30:00,4.546894,15.324332,6.423853,24.874475,14.258469,7.0,17.0,2004.0,22.0,5975.0,-77.805445,72.686592,47.977135,16.581171,0.312186
2004-09-21 19:30:00,43.385965,14.628493,5.576330,22.627802,17.359905,9.0,19.0,2004.0,21.0,83175.0,-75.000000,78.363197,47.305939,11.289606,-13.870493
2004-09-15 18:30:00,20.520223,14.666178,6.573158,19.529047,4.076474,9.0,18.0,2004.0,15.0,60150.0,-76.466973,95.035973,53.804394,6.743863,-7.025343
2004-08-26 17:30:00,8.098122,15.512402,5.839916,23.767664,5.012726,8.0,17.0,2004.0,26.0,7275.0,-73.380482,77.722290,50.499004,8.643763,10.914400
2004-09-11 18:30:00,20.549192,14.712868,4.529143,22.579614,11.788300,9.0,18.0,2004.0,11.0,6725.0,-76.211019,79.081085,42.870922,7.967304,-9.138874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-05-22 21:00:00,18.293368,15.889853,5.321009,22.611871,14.178018,5.0,21.0,2015.0,22.0,6300.0,-62.783486,85.002289,55.583439,10.845026,-13.990753
2015-06-26 20:30:00,7.232957,15.499491,6.174738,27.257166,7.789907,6.0,20.0,2015.0,26.0,15375.0,-78.557426,63.109238,44.427227,12.215550,-1.446545
2015-06-30 21:00:00,15.779857,14.543398,4.898322,27.249353,18.221504,6.0,21.0,2015.0,30.0,18750.0,-69.710906,59.499866,42.788399,12.751741,0.816761


In [19]:
for nd in node_mcs_days.keys():
    take_dates = np.intersect1d(obs['date'].values,np.array(node_mcs_days[nd]))
    out = obs_ind.loc[take_dates]
    nts = []
    for od in out.index:
        nt = od.replace(hour=0)
        nts.append(nt)
    out.index = pd.Series(nts)
   # ipdb.set_trace()
    pkl.dump(out, open(cnst.network_data + 'data/SOMS/saves/'+nd+'_MCS_matchTable.p', "wb"))

In [78]:
out

,prcp,q925,q650,t925,shear,month,hour,year,day,area,tmin,rh,tcwv,lat,lon
date,,,,,,,,,,,,,,,
2004-04-05 19:00:00,35.472010,14.413225,5.547919,26.713281,7.973417,4.0,19.0,2004.0,5.0,8900.0,-86.175891,60.574142,39.024242,7.790676,6.186016
2004-04-05 19:00:00,36.638487,14.323123,3.718602,25.902429,6.756983,4.0,19.0,2004.0,5.0,47100.0,-83.254094,63.909451,36.166225,8.355605,4.226043
2004-04-05 19:00:00,2.714505,15.481491,4.105611,25.647668,7.549984,4.0,19.0,2004.0,5.0,8300.0,-68.286055,67.925087,40.689201,7.585366,0.864516
2004-04-05 19:00:00,38.656914,16.220599,5.105718,24.308679,9.896074,4.0,19.0,2004.0,5.0,36425.0,-81.946371,79.203941,45.273964,5.939839,6.741510
2004-04-05 19:00:00,12.113163,11.405258,5.231396,31.367151,7.330038,4.0,19.0,2004.0,5.0,35000.0,-67.132884,34.169807,36.303638,7.497565,10.233313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-26 19:00:00,17.585934,17.020743,3.220877,23.947839,9.038910,4.0,19.0,2015.0,26.0,8550.0,-77.000000,84.399796,46.086990,7.582853,0.051210
2015-04-26 21:00:00,0.638106,14.362934,2.736809,27.100854,5.082678,4.0,21.0,2015.0,26.0,16500.0,-71.935047,59.512390,38.398781,8.817206,2.336917
2015-04-26 21:00:00,21.312212,15.450131,5.023412,24.569666,12.041897,4.0,21.0,2015.0,26.0,23150.0,-84.482844,73.683029,47.546383,6.887268,-3.399412
